<a href="https://colab.research.google.com/github/shivsharanrupesh/ALL_RAG/blob/main/Insurance_Chatbot_RAG/Insurance_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**RAG Evaluation with Langchain and RAGAS**

In the following notebook I will be exploring the following:

1. Creating a simple RAG pipeline with LangChain v0.1.0
2. Evaluating our pipeline with the Ragas library
3. Making an adjustment to our RAG pipeline
4. Evaluating our adjusted pipeline against our baseline

In [1]:
!pip install -q langchain langchain-openai langchain_core langchain-community langchainhub openai ragas tiktoken cohere faiss_cpu requests tokenizers pypdf2 unstructured langchain langchain_together


In [2]:
%pip install --upgrade --quiet  sentence_transformers  rank_bm25 > /dev/null

In [3]:
from openai import OpenAI
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

**Building our RAG pipeline**

I will:

1. Create an Index
2. Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.


**Loading Data**

In [5]:
from google.colab import drive

# Mount Google Drive
drive_path = drive.mount('/content/MyDrive/')

#!ls -R '/content/MyDrive/'

Drive already mounted at /content/MyDrive/; to attempt to forcibly remount, call drive.mount("/content/MyDrive/", force_remount=True).


In [6]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

#Replace with the actual path to your Markdown file in Colab:
markdown_file_path = '/content/MyDrive/MyDrive/dataset/policy-booklet-0923.md'
loader = UnstructuredMarkdownLoader(markdown_file_path)
documents = loader.load()


In [7]:
documents[0].metadata

{'source': '/content/MyDrive/MyDrive/dataset/policy-booklet-0923.md'}

**Transforming Data**

Now that I have gotten my single document - let's split it into smaller pieces so I can more effectively leverage it with our retrieval chain!

We'll start with the classic: *RecursiveCharacterTextSplitter.*

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)

Let's confirm we have split our policy document.

In [8]:
len(documents)

136

**Loading OpenAI Embeddings Model**

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use
 **OpenAI's text-embedding-ada-002**
 for this task! (soon we'll be able to leverage OpenAI's newest embedding model which is waiting on an approved PR to be merged as we speak!)

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=api_key)

<ipython-input-9-e4a8b6e740bc>:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=api_key)


**Creating a FAISS VectorStore**

Now that i have my documents - I'll need a place to store them alongside their embeddings.

I will be using Meta's FAISS for this task.

In [10]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents, embeddings)

**Creating a Retriever**

To complete my index, all that's left to do is expose my vectorstore as a retriever

In [11]:
retriever = vectorstore.as_retriever()

**Testing the Retriever**

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [12]:
retrieved_documents = retriever.invoke("how much will you pay if my car is damaged?")
for doc in retrieved_documents:
    print(doc.page_content)

Faqs How Much Will You Pay If My Car Is Damaged?

Where damage to your car is covered under your policy, we'll pay the cost of repairing or replacing your car up to its UK market value. This is the current value of your car at the time of the claim. It may be different to the amount you paid or any amount you provided when you insured your car with us.

Who Is Covered To Drive Other Cars?

Your certificate of motor insurance will show who has cover to drive other cars. We'll only cover injury to third parties, or damage caused to their property, not to the car being driven. See 'Section 1: Liability' on page 11. Am I covered if I leave my car unlocked or the keys in the car? We won't pay a claim for theft or attempted theft if your car is left:

Unlocked.

With keys or key fobs in, on, or attached to the car.

With the engine running.

With a window or roof open.

What's not included in my cover?

We don't cover things like:

Mechanical or electrical failure.

Wear and tear.
A courtesy

**Creating a RAG Chain**

**Creating a Prompt Template**

There are a few different ways I could create a prompt template - I could create a custom template, as seen in the code below, or I will simply pull a prompt from the prompt hub! Let's look at an example of that!

In [13]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple - but we'll create our own to be a bit more specific!



In [15]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

**Setting Up the Basic QA Chain**

Now we can instantiate the basic RAG chain!

I'll use LCEL directly just to see an example of it

I'll also ensure to pass-through our context - which is critical for RAGAS.

In [16]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=api_key)

In [17]:
retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

Let's test it out!

In [18]:
question = "how much will you pay if my car is damaged?"
result = retrieval_augmented_qa_chain.invoke({"question": question})
print(result["response"].content)

We'll pay the cost of repairing or replacing your car up to its UK market value.


In [19]:
question = "Are my electric car's charging cables covered?"
result = retrieval_augmented_qa_chain.invoke({"question": question})
print(result["response"].content)

Your home charger and charging cables are considered an accessory to your car. This means they're covered under 'Section 2: Fire and theft' or 'Section 4: Accidental damage' of your policy.
